In [9]:
import censusdata, pandas as pd

def import_censusdata(year, variables):
    census_data = censusdata.download('acs5', year, 
                                censusdata.censusgeo([('state','*'), ('county', '*')]), variables)

    dictionary = {'B02001_001E':'Total Pop Estimate', 'B01001_006E':'Males 15-17',\
                  'B01001A_006E':'White Males 15-17','B01001B_006E':'Black Males 15-17', 'B01001I_006E':'Hispanic Males 15-17',\
                   'B19001_001E':'HHI', 'B19001_016E':'HHI 150K-200K','B19001A_001E':'HHI 220K+'}
    census_data.rename(columns =dictionary, inplace=True)
    
    
    for index, data in census_data.iterrows():
        string_county = index.name.split(',')
        census_data.loc[index,'County'], census_data.loc[index,'State']= string_county[0], string_county[1]

       # print(string_county)
    census_data.reset_index(drop=True,inplace=True)
    census_dataframe = pd.DataFrame(census_data)

    return census_dataframe.groupby(['State', 'County']).agg({'Total Pop Estimate':'sum',
                                                                'Males 15-17':'sum', 'Black Males 15-17':'sum', 'Hispanic Males 15-17':'sum',\
                                                            'HHI 220K+':'sum'})

males_census_2019df = import_censusdata(2019, ['B02001_001E', 'B01001_006E','B01001A_006E','B01001_006E' , 'B01001B_006E', 'B01001I_006E',\
                                                'B19001_001E', 'B19001_016E','B19001A_001E'])
#males_census_2019df.sort_values(by = 'Males 15-17', ascending=False)


In [11]:
import censusdata, pandas as pd

def import_censusdata0(year, variables, geo_agg):
    census_data = censusdata.download('acs5', year, 
                                censusdata.censusgeo([('state','*'), geo_agg]), variables)

    dictionary = {'B02001_001E': 'Total Pop Estimate '+str(year), 'B19001_001E':'HHI', 'B19001_016E':'HHI 150K-200K','B19001A_001E':'HHI 220K+',\
    'B01001_006E':'Males 15-17', 'B01001A_006E':'White Males 15-17','B01001B_006E':'Black Males 15-17', 'B01001I_006E':'Hispanic Males 15-17',\
    'B01001_030E': 'Females 15-17', 'B01001A_021E':'White Females 15-17',' B01001B_021E':'Black Females 15-17','B01001I_021E': 'Hispanic Females 15-17'}
    
    census_data.rename(columns =dictionary, inplace=True)
    for index, data in census_data.iterrows():
        census_data.loc[index,'County'], census_data.loc[index,'State']= index.name, index.params()[0][1]

    #census_data.reset_index(drop=True,inplace=True)
    census_dataframe = pd.DataFrame(census_data)

    return census_dataframe#.groupby(['State', 'County']).agg({'Total Pop Estimate'+str(year):'sum',
                                                           #     'Males 15-17':'sum', 'Black Males 15-17':'sum', 'Hispanic Males 15-17':'sum',\
                                                           # 'HHI 220K+':'sum'})

male_vars = ['B02001_001E', 'B01001_006E','B01001A_006E','B01001_006E' , 'B01001B_006E', 'B01001I_006E',\
                                                'B19001_001E', 'B19001_016E','B19001A_001E']

female_vars = ['B02001_001E', 'B01001_030E', 'B01001A_021E','B01001B_021E','B01001I_021E']

males_census_2019df = import_censusdata0(2019, male_vars, ('county', '*') )
males_census_2018df = import_censusdata0(2018, male_vars, ('county', '*') )

growth_trend = pd.concat([males_census_2019df['Total Pop Estimate 2019'], males_census_2018df['Total Pop Estimate 2018']], axis=1)
growth_trend['% delta'] = growth_trend['Total Pop Estimate 2019'] / growth_trend['Total Pop Estimate 2018']


In [7]:
#this variant uses groupings for the variables
import requests
 
class census_api:
    baseurl = "https://api.census.gov/data/2019/acs/acs5?get=NAME,group(B02001)&for=zip%20code%20tabulation%20area:*&in=state:*"
    
    def __init__(self):
        #self.paramx = {'dataset'}
        self.resp = requests.get(self.baseurl).json()
         
    def output(self):
        return self.resp
census_header = census_api().output()

In [14]:
def school_data(year, dataset, var, grade):
    import urllib, json, pandas as pd
    #dir(urllib3.request.urlencode)
    if dataset == 'ccd':
         url = "https://educationdata.urban.org/api/v1/schools/"+dataset+"/"+var+"/"+str(year)+"/"
         
    elif dataset == 'crdc':
         url = "https://educationdata.urban.org/api/v1/schools/"+dataset+"/"+var+"/"+str(year)+"/"+grade+"/race/sex/"
    print(url)
    response = urllib.request.urlopen(url)
    retention = json.loads(response.read())
    return pd.json_normalize(retention, record_path= ['results'])

retention2017 = school_data(2017, 'ccd','directory', False)


https://educationdata.urban.org/api/v1/schools/ccd/directory/2017/
